<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install syft

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import syft as sy

# Define a simple model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

# Create a virtual federated learning environment
alice = sy.VirtualMachine(name="alice")
bob = sy.VirtualMachine(name="bob")
vm_workspace = sy.VirtualMachineWorkspace(
    name="workspace", vms=[alice, bob]
)

# Prepare the dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)

# Split dataset into federated chunks
federated_train_loader = sy.FederatedDataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    virtual_machines=[alice, bob],
    unique_dataset=False,
)

# Initialize the model and optimizer
model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(5):
    for data, target in federated_train_loader:
        data, target = data.to("cpu"), target.to("cpu")  # Ensure tensors are on the correct device
        model.send(data.location)
        optimizer.zero_grad()
        output = model(data)
        loss = nn.NLLLoss()(output, target)
        loss.backward()
        optimizer.step()
        model.get()
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")